In [3]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/FM_data'
file_name = 'normalized_to_rating_filter_track_5_user_50.csv'

In [4]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [13]:
n=4
num_user = len(test['uid'].unique())
num_user

218

In [14]:
for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    port = i
    predictions = pd.read_pickle(os.path.join(dir_, 'pre_filtering(new)', str(port) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))

    top_n_lists = []
    for j in range(num_user):
        prediction = predictions[predictions['uid'] == j]
        prediction = prediction.sort_values(by=['rating'],  ascending=False)
        prediction = prediction[:n]
        top_n_list = []
        for _, row in prediction.iterrows():
            top_n_list.append(row[1])
        top_n_lists.append(top_n_list)
        
    satisfication = 0
    for j in range(num_user):
        top_n_list = top_n_lists[j]
        precision = 0
        for k in top_n_list:
            p = test[test['uid'] == j]
            p = p[p['tid']==k]
            if len(p) > 0:
                precision += 1
        satisfication += precision / n
    print(satisfication/num_user)

0.11123853211009174
0.12385321100917432
0.11467889908256881
0.1158256880733945
0.12844036697247707
0.12270642201834862
0.11009174311926606
0.12155963302752294
0.11467889908256881
0.09059633027522936


In [15]:
for i in [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    port = i
    predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering(new)', str(port) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))

    top_n_lists = []
    for j in range(num_user):
        prediction = predictions[predictions['uid'] == j]
        prediction = prediction.sort_values(by=['rating'],  ascending=False)
        prediction = prediction[:n]
        top_n_list = []
        for _, row in prediction.iterrows():
            top_n_list.append(row[1])
        top_n_lists.append(top_n_list)
        
    satisfication = 0
    for j in range(num_user):
        top_n_list = top_n_lists[j]
        precision = 0
        for k in top_n_list:
            p = test[test['uid'] == j]
            p = p[p['tid']==k]
            if len(p) > 0:
                precision += 1
        satisfication += precision / n
    print(satisfication/num_user)

0.19839449541284404
0.15825688073394495
0.14908256880733944
0.12844036697247707
0.12385321100917432
0.11811926605504587
0.11697247706422019
0.11811926605504587
0.11697247706422019
0.11697247706422019
0.11697247706422019
0.11697247706422019
0.11697247706422019
0.11697247706422019


In [268]:
#############################################
#                                           #
#          Group Recommendation             #
#                                           #
#############################################

In [269]:
#############################################
#                                           #
#          Group Recommendation (2)         #
#                                           #
#############################################

In [12]:
dir_ = '../data/groups/random/'

In [21]:
groups = np.loadtxt(os.path.join(dir_, 'group_2.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([89., 39.])

In [22]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'])

In [23]:
# Prediction list of each group (2)
n=4
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
#   Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [25]:
evaluation = []
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 3.0
            high_rating = 0
            high_rating_t1 = 0
            high_rating_t2 = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(len(groups)))


0.13333333333333333


In [34]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)

    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.1197241849370243


In [210]:
#############################################
#                                           #
#          Group Recommendation (4)         #
#                                           #
#############################################

In [29]:
groups = np.loadtxt(os.path.join(dir_, 'group_4.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([102., 116.,  91., 107.])

In [30]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

In [31]:
# Prediction list of each group (4)
n=4
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    
    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['_u3','_u4'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'])/4

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [32]:
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
print(satisfication/(len(groups)))


0.375


In [46]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.14758117494863116


In [121]:
#############################################
#                                           #
#          Group Recommendation (5)         #
#                                           #
#############################################

In [38]:
groups = np.loadtxt(os.path.join(dir_, 'group_5.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([18., 24.,  1., 54., 90.])

In [39]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

In [40]:
# Prediction list of each group (5)
n=4
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5, on=['tid'], how='inner', suffixes=['_u3','_u5'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'])/5
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [41]:
evaluation = []
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(len(groups)))


0.3645833333333333


In [37]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    high_rating_u5 = 0

    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]

        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1                
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))

AttributeError: type object 'tqdm_notebook' has no attribute 'tqdm_notebook'

In [ ]:
#############################################
#                                           #
#          Group Recommendation (10)        #
#                                           #
#############################################

In [58]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_10_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 17.,  58.,  73., 104., 112., 164., 207., 214., 218., 219.])

In [59]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

def least_misery(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

In [60]:
# Prediction list of each group (10)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]
    u6 = predictions[predictions['uid'] == group[5]]
    u7 = predictions[predictions['uid'] == group[6]]
    u8 = predictions[predictions['uid'] == group[7]]
    u9 = predictions[predictions['uid'] == group[8]]
    u10 = predictions[predictions['uid'] == group[9]]

    m = pd.merge(u1, u2,  on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3,  on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4,  on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5,  on=['tid'], how='inner', suffixes=['','_u5'])
    m = pd.merge(m,  u6,  on=['tid'], how='inner', suffixes=['','_u6'])
    m = pd.merge(m,  u7,  on=['tid'], how='inner', suffixes=['','_u7'])
    m = pd.merge(m,  u8,  on=['tid'], how='inner', suffixes=['','_u8'])
    m = pd.merge(m,  u9,  on=['tid'], how='inner', suffixes=['','_u9'])
    m = pd.merge(m,  u10, on=['tid'], how='inner', suffixes=['_u3','_u10'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5', 'rating_u6','rating_u7','rating_u8','rating_u9','rating_u10']]

#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'] + m['rating_u6'] + m['rating_u7'] + m['rating_u8'] + m['rating_u9'] + + m['rating_u10'])/10
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [162]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 1.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
#             precision += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.7159090909090909


In [65]:
# modified satisfaction
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    satisfication_t = 0
    satisfication_u = 0
    high_rating_u1 = 0
    high_rating_u2 = 0
    high_rating_u3 = 0
    high_rating_u4 = 0
    high_rating_u5 = 0
    high_rating_u6 = 0
    high_rating_u7 = 0
    high_rating_u8 = 0
    high_rating_u9 = 0
    high_rating_u10 = 0

    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            rating = 4.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u1 += 1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u2 += 1
            if len(t3) > 0 and t3.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u3 += 1
            if len(t4) > 0 and t4.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u4 += 1
            if len(t5) > 0 and t5.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u5 += 1  
            if len(t6) > 0 and t6.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u6 += 1  
            if len(t7) > 0 and t7.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u7 += 1  
            if len(t8) > 0 and t8.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u8 += 1  
            if len(t9) > 0 and t9.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u9 += 1  
            if len(t10) > 0 and t10.iloc[0]['rating'] > rating:
                high_rating +=1
                high_rating_u10 += 1  
            satisfication_t += math.log(high_rating+1,10)/math.log(len(group)+1,10)
    satisfication_u += math.log(high_rating_u1+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u2+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u3+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u4+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u5+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u6+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u7+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u8+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u9+1,10)/math.log(n+1,10)
    satisfication_u += math.log(high_rating_u10+1,10)/math.log(n+1,10)
    satisfication += 0.5 * satisfication_t / n
    satisfication += 0.5 * satisfication_u / len(group)
print(satisfication/(num_user//group_size))


0.1695292296740876


In [326]:
groups = []
group_size = 10

u_del = []
num_u_del = num_user - (num_user//group_size)*group_size

for i in range(num_u_del):
    u = random.random()*num_user//1
    u_del.append(u)

for i in range(num_user//group_size):
    groups.append([])
    
for i in range(num_user):
    b = True
    
    if i in u_del:
        b = False
        
    while b:
        g = random.random()*(num_user/group_size)//1
        if int(g) < len(groups):
            group = groups[int(g)]
            if len(group) < group_size:
                group.append(i)
                b = False
groups

[[28, 51, 73, 87, 99, 113, 117, 118, 136, 137],
 [10, 13, 20, 21, 39, 50, 110, 119, 128, 177],
 [16, 32, 58, 77, 88, 127, 159, 179, 183, 189],
 [2, 3, 7, 29, 30, 34, 36, 47, 57, 66],
 [6, 68, 76, 98, 103, 120, 125, 139, 157, 172],
 [44, 75, 121, 173, 196, 203, 206, 207, 209, 210],
 [0, 9, 65, 67, 70, 104, 106, 144, 150, 156],
 [4, 27, 40, 56, 74, 83, 91, 97, 100, 105],
 [17, 18, 24, 37, 46, 146, 149, 175, 176, 192],
 [22, 60, 80, 81, 131, 133, 135, 138, 155, 164],
 [25, 26, 33, 49, 130, 171, 184, 194, 197, 200],
 [14, 41, 52, 62, 63, 64, 84, 86, 102, 108],
 [61, 71, 90, 116, 129, 132, 165, 185, 190, 195],
 [38, 45, 85, 93, 140, 147, 161, 181, 191, 198],
 [12, 42, 89, 92, 96, 107, 142, 188, 205, 208],
 [43, 53, 101, 112, 126, 152, 163, 182, 186, 187],
 [1, 19, 23, 48, 78, 115, 123, 145, 148, 167],
 [35, 54, 55, 69, 109, 124, 141, 153, 158, 170],
 [15, 31, 82, 95, 151, 193, 199, 201, 202, 204],
 [5, 79, 134, 143, 160, 162, 166, 168, 178, 180],
 [8, 11, 59, 94, 111, 114, 122, 154, 169, 17

In [327]:
# check if duplicate 

users = []
for group in groups:
    for u in group:
        users.append(u)
users.sort(reverse=True)

n = num_user - 1 
for i in users:
    if i-n != 0:
        print(i, 'False')
        n-=1
    n-=1

71 False


In [328]:
np.savetxt(os.path.join(dir_, 'groups_' + str(group_size) + '_' + file_name[:-3] + 'csv'), groups, delimiter=",")